In [1]:
import tensorflow as tf
import os

In [2]:
model = tf.keras.models.load_model("data01_20200521_46epochs.h5")

In [3]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 60, 200, 32)  896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 58, 198, 32)  9248        conv2d_7[0][0]                   
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 58, 198, 32)  128         conv2d_8[0][0]                   
____________________________________________________________________________________________

In [4]:
#no crop function
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
        #crop to [60,30,3]
    img = tf.io.decode_jpeg(a)
    return img

val_dir_data01 = os.path.abspath(os.getcwd())+"\\dev\\data01_dev"
#validation data no crop
val_dir = tf.data.Dataset.list_files(val_dir_data01+'*.jpg',shuffle=False) 
val_data01 = val_dir.map(lambda x: readimg_to_tensor(x))

In [5]:
import numpy as np
import pandas as pd

In [6]:

CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [7]:
def parseResult(RS):
    for j in range(len(RS[0])):
        PredChr=''
        for i in range(6):
            PredChr+= CLASS_NAMES[(np.where(RS[i][j] == RS[i][j].max())[0][0])]
        if j == 0:
            LB = [PredChr]
        else:
            LB.append(PredChr)      
    return LB
#output to CSV
VDlabel = np.array([])
img = next(iter(val_data01.batch(10000)))
result = model.predict(img,use_multiprocessing = True)
val_dir = tf.data.Dataset.list_files(val_dir_data01+'*.jpg',shuffle=False) 
test = parseResult(result)
val_file_name = pd.read_csv("./dev/data01_dev.csv")
val_file_name.join(pd.DataFrame(test,columns=["code"])).to_csv("data01_dev.csv",index = False)      